<center>

![huawei-logo](https://www-file.huawei.com/-/media/corporate/images/home/logo/huawei_logo.png)

<p><b>Летняя школа. "Современные методы теории информации, оптимизации и управления".</p></b>
Sirius-2020.
<center> <b>Авторы материала: к.т.н. Антонов Лев, Власов Роман.

## Калассификация реальных данных с помощью Autokeras

In [ ]:
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc1 &> /dev/null
!pip install autokeras &> /dev/null

Теперь давайде возьмем боевой датасет с Kaggle и попробуем в автоматическом режиме синтезировать модель.

В качестве набора данных возьмем датасет конкурса **"Santander Customer Transaction Prediction"**. Данный датасет хорош тем, что признаки уже очищены от пропущенных значений. Все данные обезличены и приведены к числовому формату. Можно использовать данный датасет для проверки фреймворка AutoKeras.


https://www.kaggle.com/c/santander-customer-transaction-prediction

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
import autokeras as ak
import kerastuner as kt
import scipy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/datasets/train.csv')
data = pd.read_csv('../../../../../_datasets/SantanderCusyomerTransactionPrediction/train.csv')
data.head(5)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [ ]:
data = data.drop(columns=['ID_code'], axis=1)
data.shape

(200000, 201)

In [ ]:
train_data = data[:180000]
val_data   = data[180000:]

In [ ]:
OUTPUT_PATH = '4_Santander_Classification'

In [ ]:
x_train = train_data.drop(columns=['target'], axis=1)
y_train = train_data['target']
x_val   = val_data.drop(columns=['target'], axis=1)
y_val   = val_data['target']

#### Симуляция 1

Для начального эксперимента установим **max_trials=2** и количество эпох для обучения модели epochs=10.



In [ ]:
clf = ak.StructuredDataClassifier(num_classes=2, 
                                  max_trials=2, 
                                  objective=kt.Objective("val_auc", direction="max"), 
                                  metrics=["AUC", "accuracy"], 
                                  directory=OUTPUT_PATH)

In [ ]:
clf.fit(x_train,
        y_train,
        # Use your own validation set.
        validation_data=(x_val, y_val),
        epochs=10)

Trial 2 Complete [00h 04m 03s]
val_auc: 0.8282268643379211

Best val_auc So Far: 0.8468431830406189
Total elapsed time: 00h 08m 42s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
5625/5625 [==============================] - 26s 5ms/step - loss: 0.2746 - auc: 0.7845 - accuracy: 0.9012 - val_loss: 0.2760 - val_auc: 0.8087 - val_accuracy: 0.9038
Epoch 2/10
5625/5625 [==============================] - 26s 5ms/step - loss: 0.2504 - auc: 0.8321 - accuracy: 0.9071 - val_loss: 0.2486 - val_auc: 0.8476 - val_accuracy: 0.9087
Epoch 3/10
5625/5625 [==============================] - 26s 5ms/step - loss: 0.2474 - auc: 0.8371 - accuracy: 0.9080 - val_loss: 0.2561 - val_auc: 0.8321 - val_accuracy: 0.9078
Epoch 4/10
5625/5625 [==============================] - 26s 5ms/step - loss: 0.2458 - auc: 0.8399 - accuracy: 0.9080 - val_loss: 0.2618 - val_auc: 0.8315 - val_accuracy: 0.9066
Epoch 5/10
5625/5625 [==============================] - 26s 5ms/step - loss: 0.2445 - auc: 0.8422 - accuracy: 0.9087 - val

In [ ]:
model = clf.export_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                6432      
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)               

In [ ]:
#x_val_ = tf.convert_to_tensor(x_val, dtype=tf.float32)
y_pred = clf.predict(x_val)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score
accuracy_score(y_pred, y_val)

0.9119

In [ ]:
roc_auc_score(y_pred, y_val)

0.8232198541382778

#### Симуляция 2

Увеличим время ожидания архитектруного синтеза.

In [ ]:
OUTPUT_PATH = '4_Santander_Classification_10'

In [ ]:
clf = ak.StructuredDataClassifier(num_classes=2, 
                                  max_trials=25, 
                                  objective=kt.Objective("val_auc", direction="max"), 
                                  metrics=["AUC", "accuracy"], 
                                  directory=OUTPUT_PATH)

In [ ]:
%%time
clf.fit(x_train,
        y_train,
        # Use your own validation set.
        validation_data=(x_val, y_val),
        epochs=30)

Trial 25 Complete [00h 10m 34s]
val_auc: 0.5

Best val_auc So Far: 0.8603365421295166
Total elapsed time: 06h 49m 44s
INFO:tensorflow:Oracle triggered exit
Epoch 1/30
5625/5625 [==============================] - 39s 7ms/step - loss: 0.2921 - auc: 0.7442 - accuracy: 0.9001 - val_loss: 0.2539 - val_auc: 0.8341 - val_accuracy: 0.9054
Epoch 2/30
5625/5625 [==============================] - 38s 7ms/step - loss: 0.2623 - auc: 0.8097 - accuracy: 0.9043 - val_loss: 0.2538 - val_auc: 0.8394 - val_accuracy: 0.9064
Epoch 3/30
5625/5625 [==============================] - 38s 7ms/step - loss: 0.2568 - auc: 0.8196 - accuracy: 0.9059 - val_loss: 0.2487 - val_auc: 0.8457 - val_accuracy: 0.9067
Epoch 4/30
5625/5625 [==============================] - 38s 7ms/step - loss: 0.2533 - auc: 0.8263 - accuracy: 0.9068 - val_loss: 0.2420 - val_auc: 0.8519 - val_accuracy: 0.9092
Epoch 5/30
5625/5625 [==============================] - 39s 7ms/step - loss: 0.2510 - auc: 0.8302 - accuracy: 0.9074 - val_loss: 0.2443 

In [ ]:
model = clf.export_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               102912    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
re_lu (ReLU)                 (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [ ]:
clf.evaluate(x_val, y_val)

625/625 [==============================] - 3s 4ms/step - loss: 0.2351 - auc: 0.8588 - accuracy: 0.9134


[0.23506708443164825, 0.8587598204612732, 0.9133999943733215]

Как видим, за счет довольно большого времени ожидания смогли улучшить результат классификации на **3,56%!**

Теперь давайте немного поработаем с признаками:
1. Cначала проведем их стандартизацию.
2. Попробуем насемплировать новых признаков с помощью их линейных комбинаций (а вдруг поможет?).

In [ ]:
data_x =  data.drop(columns=['target'], axis=1)
data_y =  data['target']
data_x.shape, data_y.shape

((200000, 200), (200000,))

In [ ]:
from sklearn import  preprocessing
st_data_x = preprocessing.scale(data_x)
pd.DataFrame(st_data_x).head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.577102,-1.273737,0.451707,-0.833709,0.235571,-0.536430,-0.334926,0.608751,-1.561580,-1.473796,...,0.263374,-1.149158,0.817469,-0.411013,0.168705,-1.578117,1.022131,-0.373968,-1.026398,0.214135
1,0.269959,-0.622138,1.190360,-0.688846,0.790975,1.539900,0.244461,-0.003525,0.858974,0.419300,...,0.966611,0.093605,0.443623,1.908764,-0.817594,1.522342,1.067654,-0.129400,0.825417,0.505685
2,-0.681113,-0.276066,0.516988,0.536516,-0.305477,-0.511033,1.769839,-0.564749,-1.561370,-1.307408,...,-0.072093,0.777997,-0.174131,-0.412316,1.151591,2.297370,-1.617906,-0.695141,-0.381449,0.356681
3,0.125158,-0.129426,-0.667575,0.195355,0.927992,0.410672,0.500633,-0.474201,-1.843910,0.548767,...,0.270216,-0.891456,-0.818468,-0.478548,1.607869,-0.789517,-0.959020,1.501744,0.697118,-0.543502
4,-0.277303,0.035610,0.817683,-0.077829,0.738607,0.955574,0.613372,0.791544,1.794753,0.090006,...,-1.036191,0.688988,-1.405987,1.468536,-1.501101,-0.958473,0.297627,0.645537,0.706318,-0.525375


Разбиваем выборку на train и validation.

In [ ]:
x_train_st = st_data_x[:180000]
y_train = data_y[:180000]
x_val_st   = st_data_x[180000:]
y_val = data_y[180000:]
x_train_st.shape, y_train.shape, x_val_st.shape, y_val.shape

((180000, 200), (180000,), (20000, 200), (20000,))

#### Симуляция 3

Запустим классификатор с малым временем ожидания, чтобы оценить эффект от преобразования данных: установим max_trials=2 и количество эпох для обучения модели epochs=10.

In [ ]:
OUTPUT_PATH = '4_Santander_Classification_st_data'
clf = ak.StructuredDataClassifier(num_classes=2, 
                                  max_trials=2, 
                                  objective=kt.Objective("val_auc", direction="max"), 
                                  metrics=["AUC", "accuracy"], 
                                  directory=OUTPUT_PATH)

In [ ]:
clf.fit(x_train_st,
        y_train,
        # Use your own validation set.
        validation_data=(x_val_st, y_val),
        epochs=10)

Epoch 1/10


W0812 13:20:21.961438 4399074752 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


1718/5625 [========>.....................] - ETA: 0s - loss: 43.0408 - auc: 0.5000 - accuracy: 0.937 - ETA: 26s - loss: 39.1202 - auc: 0.5411 - accuracy: 0.83 - ETA: 25s - loss: 39.6839 - auc: 0.5337 - accuracy: 0.83 - ETA: 24s - loss: 35.0938 - auc: 0.5400 - accuracy: 0.82 - ETA: 23s - loss: 33.6591 - auc: 0.5349 - accuracy: 0.83 - ETA: 23s - loss: 36.1724 - auc: 0.5262 - accuracy: 0.82 - ETA: 23s - loss: 34.4898 - auc: 0.5293 - accuracy: 0.82 - ETA: 23s - loss: 33.9243 - auc: 0.5319 - accuracy: 0.82 - ETA: 23s - loss: 32.7329 - auc: 0.5272 - accuracy: 0.82 - ETA: 23s - loss: 31.9122 - auc: 0.5255 - accuracy: 0.82 - ETA: 24s - loss: 31.6269 - auc: 0.5247 - accuracy: 0.82 - ETA: 24s - loss: 30.7636 - auc: 0.5243 - accuracy: 0.82 - ETA: 24s - loss: 30.1587 - auc: 0.5249 - accuracy: 0.82 - ETA: 24s - loss: 29.1155 - auc: 0.5277 - accuracy: 0.82 - ETA: 23s - loss: 28.4871 - auc: 0.5263 - accuracy: 0.82 - ETA: 23s - loss: 27.7208 - auc: 0.5265 - accuracy: 0.83 - ETA: 23s - loss: 27.5844 - 

3412/5625 [=================>............] - ETA: 16s - loss: 5.8679 - auc: 0.5158 - accuracy: 0.844 - ETA: 16s - loss: 5.8273 - auc: 0.5155 - accuracy: 0.844 - ETA: 16s - loss: 5.7872 - auc: 0.5157 - accuracy: 0.844 - ETA: 16s - loss: 5.7506 - auc: 0.5154 - accuracy: 0.845 - ETA: 16s - loss: 5.7243 - auc: 0.5154 - accuracy: 0.845 - ETA: 16s - loss: 5.6971 - auc: 0.5155 - accuracy: 0.845 - ETA: 16s - loss: 5.6648 - auc: 0.5158 - accuracy: 0.845 - ETA: 16s - loss: 5.6306 - auc: 0.5159 - accuracy: 0.846 - ETA: 16s - loss: 5.5962 - auc: 0.5161 - accuracy: 0.846 - ETA: 16s - loss: 5.5627 - auc: 0.5160 - accuracy: 0.846 - ETA: 16s - loss: 5.5241 - auc: 0.5160 - accuracy: 0.846 - ETA: 15s - loss: 5.4857 - auc: 0.5157 - accuracy: 0.847 - ETA: 15s - loss: 5.4507 - auc: 0.5158 - accuracy: 0.847 - ETA: 15s - loss: 5.4188 - auc: 0.5156 - accuracy: 0.847 - ETA: 15s - loss: 5.3921 - auc: 0.5157 - accuracy: 0.847 - ETA: 15s - loss: 5.3606 - auc: 0.5155 - accuracy: 0.848 - ETA: 15s - loss: 5.3329 - a

5172/5625 [==========================>...] - ETA: 9s - loss: 3.1543 - auc: 0.5204 - accuracy: 0.86 - ETA: 9s - loss: 3.1436 - auc: 0.5205 - accuracy: 0.86 - ETA: 9s - loss: 3.1332 - auc: 0.5205 - accuracy: 0.86 - ETA: 9s - loss: 3.1227 - auc: 0.5206 - accuracy: 0.86 - ETA: 9s - loss: 3.1123 - auc: 0.5207 - accuracy: 0.86 - ETA: 9s - loss: 3.1018 - auc: 0.5207 - accuracy: 0.86 - ETA: 9s - loss: 3.0914 - auc: 0.5209 - accuracy: 0.86 - ETA: 9s - loss: 3.0812 - auc: 0.5213 - accuracy: 0.86 - ETA: 8s - loss: 3.0725 - auc: 0.5217 - accuracy: 0.86 - ETA: 8s - loss: 3.0635 - auc: 0.5217 - accuracy: 0.86 - ETA: 8s - loss: 3.0536 - auc: 0.5220 - accuracy: 0.86 - ETA: 8s - loss: 3.0436 - auc: 0.5219 - accuracy: 0.87 - ETA: 8s - loss: 3.0338 - auc: 0.5223 - accuracy: 0.87 - ETA: 8s - loss: 3.0239 - auc: 0.5225 - accuracy: 0.87 - ETA: 8s - loss: 3.0141 - auc: 0.5226 - accuracy: 0.87 - ETA: 8s - loss: 3.0052 - auc: 0.5226 - accuracy: 0.87 - ETA: 8s - loss: 2.9965 - auc: 0.5229 - accuracy: 0.87 - ETA

...

5625/5625 [==============================] - ETA: 3s - loss: 0.2794 - auc: 0.7625 - accuracy: 0.90 - ETA: 3s - loss: 0.2794 - auc: 0.7628 - accuracy: 0.90 - ETA: 3s - loss: 0.2794 - auc: 0.7628 - accuracy: 0.90 - ETA: 3s - loss: 0.2793 - auc: 0.7630 - accuracy: 0.90 - ETA: 2s - loss: 0.2793 - auc: 0.7630 - accuracy: 0.90 - ETA: 2s - loss: 0.2793 - auc: 0.7630 - accuracy: 0.90 - ETA: 2s - loss: 0.2792 - auc: 0.7630 - accuracy: 0.90 - ETA: 2s - loss: 0.2792 - auc: 0.7630 - accuracy: 0.90 - ETA: 2s - loss: 0.2791 - auc: 0.7630 - accuracy: 0.90 - ETA: 2s - loss: 0.2790 - auc: 0.7630 - accuracy: 0.90 - ETA: 2s - loss: 0.2791 - auc: 0.7630 - accuracy: 0.90 - ETA: 2s - loss: 0.2790 - auc: 0.7631 - accuracy: 0.90 - ETA: 2s - loss: 0.2789 - auc: 0.7632 - accuracy: 0.90 - ETA: 2s - loss: 0.2788 - auc: 0.7634 - accuracy: 0.90 - ETA: 2s - loss: 0.2788 - auc: 0.7636 - accuracy: 0.90 - ETA: 2s - loss: 0.2788 - auc: 0.7637 - accuracy: 0.90 - ETA: 2s - loss: 0.2789 - auc: 0.7638 - accuracy: 0.90 - ETA

1630/5625 [=======>......................] - ETA: 0s - loss: 0.1657 - auc: 0.8750 - accuracy: 0.96 - ETA: 24s - loss: 0.2582 - auc: 0.7727 - accuracy: 0.911 - ETA: 26s - loss: 0.2919 - auc: 0.7211 - accuracy: 0.900 - ETA: 28s - loss: 0.2832 - auc: 0.7384 - accuracy: 0.903 - ETA: 28s - loss: 0.2826 - auc: 0.7381 - accuracy: 0.904 - ETA: 27s - loss: 0.2725 - auc: 0.7503 - accuracy: 0.906 - ETA: 29s - loss: 0.2780 - auc: 0.7487 - accuracy: 0.904 - ETA: 28s - loss: 0.2784 - auc: 0.7549 - accuracy: 0.904 - ETA: 27s - loss: 0.2798 - auc: 0.7607 - accuracy: 0.902 - ETA: 26s - loss: 0.2754 - auc: 0.7567 - accuracy: 0.904 - ETA: 26s - loss: 0.2757 - auc: 0.7595 - accuracy: 0.903 - ETA: 25s - loss: 0.2728 - auc: 0.7691 - accuracy: 0.903 - ETA: 25s - loss: 0.2742 - auc: 0.7633 - accuracy: 0.903 - ETA: 25s - loss: 0.2714 - auc: 0.7653 - accuracy: 0.904 - ETA: 25s - loss: 0.2690 - auc: 0.7690 - accuracy: 0.905 - ETA: 24s - loss: 0.2682 - auc: 0.7653 - accuracy: 0.907 - ETA: 24s - loss: 0.2713 - auc

3260/5625 [================>.............] - ETA: 17s - loss: 0.2757 - auc: 0.7676 - accuracy: 0.904 - ETA: 17s - loss: 0.2756 - auc: 0.7682 - accuracy: 0.904 - ETA: 17s - loss: 0.2762 - auc: 0.7679 - accuracy: 0.904 - ETA: 17s - loss: 0.2758 - auc: 0.7685 - accuracy: 0.904 - ETA: 17s - loss: 0.2759 - auc: 0.7685 - accuracy: 0.904 - ETA: 17s - loss: 0.2758 - auc: 0.7680 - accuracy: 0.904 - ETA: 17s - loss: 0.2759 - auc: 0.7680 - accuracy: 0.904 - ETA: 17s - loss: 0.2754 - auc: 0.7681 - accuracy: 0.904 - ETA: 17s - loss: 0.2755 - auc: 0.7681 - accuracy: 0.904 - ETA: 17s - loss: 0.2754 - auc: 0.7678 - accuracy: 0.904 - ETA: 17s - loss: 0.2751 - auc: 0.7680 - accuracy: 0.904 - ETA: 17s - loss: 0.2750 - auc: 0.7683 - accuracy: 0.904 - ETA: 17s - loss: 0.2750 - auc: 0.7689 - accuracy: 0.904 - ETA: 17s - loss: 0.2753 - auc: 0.7685 - accuracy: 0.904 - ETA: 17s - loss: 0.2755 - auc: 0.7688 - accuracy: 0.904 - ETA: 16s - loss: 0.2755 - auc: 0.7695 - accuracy: 0.904 - ETA: 16s - loss: 0.2754 - a

4787/5625 [========================>.....] - ETA: 10s - loss: 0.2778 - auc: 0.7691 - accuracy: 0.903 - ETA: 10s - loss: 0.2778 - auc: 0.7693 - accuracy: 0.903 - ETA: 10s - loss: 0.2776 - auc: 0.7693 - accuracy: 0.903 - ETA: 10s - loss: 0.2775 - auc: 0.7695 - accuracy: 0.903 - ETA: 10s - loss: 0.2776 - auc: 0.7693 - accuracy: 0.903 - ETA: 10s - loss: 0.2775 - auc: 0.7694 - accuracy: 0.903 - ETA: 10s - loss: 0.2775 - auc: 0.7693 - accuracy: 0.903 - ETA: 10s - loss: 0.2774 - auc: 0.7691 - accuracy: 0.903 - ETA: 10s - loss: 0.2773 - auc: 0.7691 - accuracy: 0.903 - ETA: 10s - loss: 0.2772 - auc: 0.7690 - accuracy: 0.903 - ETA: 10s - loss: 0.2772 - auc: 0.7690 - accuracy: 0.903 - ETA: 10s - loss: 0.2770 - auc: 0.7689 - accuracy: 0.903 - ETA: 10s - loss: 0.2770 - auc: 0.7690 - accuracy: 0.903 - ETA: 10s - loss: 0.2771 - auc: 0.7690 - accuracy: 0.903 - ETA: 10s - loss: 0.2772 - auc: 0.7689 - accuracy: 0.903 - ETA: 9s - loss: 0.2773 - auc: 0.7691 - accuracy: 0.903 - ETA: 9s - loss: 0.2772 - auc

5625/5625 [==============================] - ETA: 3s - loss: 0.2761 - auc: 0.7710 - accuracy: 0.90 - ETA: 3s - loss: 0.2764 - auc: 0.7707 - accuracy: 0.90 - ETA: 3s - loss: 0.2764 - auc: 0.7708 - accuracy: 0.90 - ETA: 3s - loss: 0.2763 - auc: 0.7708 - accuracy: 0.90 - ETA: 3s - loss: 0.2763 - auc: 0.7708 - accuracy: 0.90 - ETA: 3s - loss: 0.2762 - auc: 0.7709 - accuracy: 0.90 - ETA: 3s - loss: 0.2764 - auc: 0.7708 - accuracy: 0.90 - ETA: 3s - loss: 0.2764 - auc: 0.7708 - accuracy: 0.90 - ETA: 3s - loss: 0.2764 - auc: 0.7708 - accuracy: 0.90 - ETA: 3s - loss: 0.2764 - auc: 0.7709 - accuracy: 0.90 - ETA: 3s - loss: 0.2764 - auc: 0.7710 - accuracy: 0.90 - ETA: 3s - loss: 0.2763 - auc: 0.7711 - accuracy: 0.90 - ETA: 3s - loss: 0.2764 - auc: 0.7710 - accuracy: 0.90 - ETA: 3s - loss: 0.2765 - auc: 0.7708 - accuracy: 0.90 - ETA: 3s - loss: 0.2764 - auc: 0.7709 - accuracy: 0.90 - ETA: 3s - loss: 0.2764 - auc: 0.7710 - accuracy: 0.90 - ETA: 3s - loss: 0.2765 - auc: 0.7710 - accuracy: 0.90 - ETA

In [ ]:
model = clf.export_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
multi_column_categorical_enc (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               102912    
_________________________________________________________________
re_lu (ReLU)                 (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
_________________________________________________________________
classification_head_1 (Activ (None, 1)                 0     

In [ ]:
#x_val_ = tf.convert_to_tensor(x_val, dtype=tf.float32)
y_pred = clf.predict(x_val_st)

W0812 14:04:05.453068 4399074752 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0812 14:04:05.455390 4399074752 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0812 14:04:05.457206 4399074752 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0812 14:04:05.459038 4399074752 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0812 14:04:05.461223 4399074752 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0812 14:04:05.462952 4399074752 util.py:152] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/guide/checkpoint#loading_mechanics for details.
W0812 

In [ ]:
clf.evaluate(x_val_st, y_val)

W0812 14:04:20.247890 4399074752 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0812 14:04:20.250256 4399074752 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0812 14:04:20.252163 4399074752 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0812 14:04:20.254387 4399074752 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0812 14:04:20.256666 4399074752 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0812 14:04:20.259054 4399074752 util.py:152] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/guide/checkpoint#loading_mechanics for details.
W0812 

625/625 [==============================] - ETA: 0s - loss: 0.2590 - auc: 0.9554 - accuracy: 0.87 - ETA: 1s - loss: 0.2400 - auc: 0.8938 - accuracy: 0.90 - ETA: 1s - loss: 0.2503 - auc: 0.8796 - accuracy: 0.89 - ETA: 1s - loss: 0.2677 - auc: 0.8634 - accuracy: 0.89 - ETA: 1s - loss: 0.2635 - auc: 0.8578 - accuracy: 0.89 - ETA: 1s - loss: 0.2647 - auc: 0.8580 - accuracy: 0.89 - ETA: 1s - loss: 0.2638 - auc: 0.8604 - accuracy: 0.89 - ETA: 1s - loss: 0.2583 - auc: 0.8566 - accuracy: 0.90 - ETA: 1s - loss: 0.2593 - auc: 0.8572 - accuracy: 0.89 - ETA: 1s - loss: 0.2582 - auc: 0.8575 - accuracy: 0.90 - ETA: 1s - loss: 0.2575 - auc: 0.8582 - accuracy: 0.90 - ETA: 1s - loss: 0.2572 - auc: 0.8594 - accuracy: 0.90 - ETA: 1s - loss: 0.2596 - auc: 0.8596 - accuracy: 0.89 - ETA: 1s - loss: 0.2603 - auc: 0.8586 - accuracy: 0.89 - ETA: 1s - loss: 0.2631 - auc: 0.8565 - accuracy: 0.89 - ETA: 1s - loss: 0.2615 - auc: 0.8579 - accuracy: 0.89 - ETA: 1s - loss: 0.2623 - auc: 0.8543 - accuracy: 0.89 - ETA: 

[0.2642332911491394, 0.8398925065994263, 0.8990499973297119]

Резюмируем результаты классификации:

| Значение параметров        | Результаты (roc_auc) |Время архитектурного поиска|
| :------------------------- | ---------------------- | --------- |
| max_trials = 2, epochs = 10, not norm dataset| 0.8282                 |00h 08m 42s|
| max_trials = 25, epochs =30, not norm dataset| 0.8587                   |06h 49m 44s|
| max_trials = 2, epochs = 10, norm dataset| 0.8398              |~00h 20m 00s cpu Macbook 2017|